# ECH Workers Android 客户端编译

## 功能
- ECH (Encrypted Client Hello) 支持
- WebSocket + Yamux 多路复用
- TLS 1.3
- 全局 VPN 代理 (gVisor TUN → SOCKS5)
- 分应用代理（白名单/黑名单）
- 连接测试 + 延迟显示

## 1. 安装 Go 1.24+

In [ ]:
%%bash
GO_VERSION="1.24.3"
wget -q https://go.dev/dl/go${GO_VERSION}.linux-amd64.tar.gz
sudo rm -rf /usr/local/go
sudo tar -C /usr/local -xzf go${GO_VERSION}.linux-amd64.tar.gz
rm go${GO_VERSION}.linux-amd64.tar.gz
/usr/local/go/bin/go version

In [ ]:
import os
os.environ['GOROOT'] = '/usr/local/go'
os.environ['GOPATH'] = '/root/go'
os.environ['PATH'] = f"/usr/local/go/bin:/root/go/bin:{os.environ['PATH']}"
!go version

## 2. 安装 Android SDK 和 NDK

In [ ]:
%%bash
CMDLINE_TOOLS_VERSION="11076708"
ANDROID_HOME="/root/android-sdk"

mkdir -p $ANDROID_HOME/cmdline-tools
cd $ANDROID_HOME/cmdline-tools
wget -q https://dl.google.com/android/repository/commandlinetools-linux-${CMDLINE_TOOLS_VERSION}_latest.zip
unzip -q commandlinetools-linux-${CMDLINE_TOOLS_VERSION}_latest.zip
mv cmdline-tools latest
rm commandlinetools-linux-${CMDLINE_TOOLS_VERSION}_latest.zip
echo "Android SDK 命令行工具已安装"

In [ ]:
import os
os.environ['ANDROID_HOME'] = '/root/android-sdk'
os.environ['ANDROID_SDK_ROOT'] = '/root/android-sdk'
os.environ['PATH'] = f"/root/android-sdk/cmdline-tools/latest/bin:/root/android-sdk/platform-tools:{os.environ['PATH']}"
print("Android 环境变量已设置")

In [ ]:
%%bash
yes | sdkmanager --licenses > /dev/null 2>&1
sdkmanager --install \
    "platform-tools" \
    "platforms;android-34" \
    "build-tools;34.0.0" \
    "ndk;26.1.10909125"
echo "Android SDK 组件已安装"

In [ ]:
import os
os.environ['ANDROID_NDK_HOME'] = '/root/android-sdk/ndk/26.1.10909125'
os.environ['NDK'] = '/root/android-sdk/ndk/26.1.10909125'
print(f"NDK: {os.environ['ANDROID_NDK_HOME']}")

## 3. 安装 gomobile

In [ ]:
%%bash
# 完全清理
rm -rf /root/go/bin/gomobile /root/go/bin/gobind
rm -rf /root/go/pkg/mod/golang.org/x/mobile*

# 设置 GO111MODULE
export GO111MODULE=on

# 从源码构建 gomobile（而不是 go install）
cd /tmp
git clone https://go.googlesource.com/mobile
cd mobile
go build -o /root/go/bin/gomobile ./cmd/gomobile
go build -o /root/go/bin/gobind ./cmd/gobind

# 初始化
/root/go/bin/gomobile init

echo "=== 验证 ==="
ls -la /root/go/bin/gomobile
ls -la /root/go/bin/gobind

In [ ]:
%%bash
cd /content
rm -rf ech-workers-NG
git clone https://github.com/justinwoo280/ech-workers-NG.git
echo "项目已克隆"
ls -la ech-workers-NG/

## 5. 编译 Go 核心库 (AAR)

In [ ]:
%%bash
cd /content/ech-workers-NG/core

# 下载依赖
go mod tidy

echo "Go 依赖已下载"
cat go.mod

In [ ]:
%%bash
cd /content/ech-workers-NG/core
export GOFLAGS="-mod=mod"
mkdir -p ../app/libs
gomobile bind -v -target=android -androidapi 21 -o ../app/libs/core.aar .
ls -lh ../app/libs/

## 6. 编译 Debug APK

In [ ]:
%%bash
cd /content/ech-workers-NG

# 修复 Windows 换行符问题 (CRLF -> LF)
echo "=== 修复换行符 ==="
sed -i 's/\r$//' gradlew
sed -i 's/\r$//' gradle.properties
sed -i 's/\r$//' settings.gradle.kts
sed -i 's/\r$//' build.gradle.kts
sed -i 's/\r$//' app/build.gradle.kts
find . -name "*.properties" -exec sed -i 's/\r$//' {} \;

chmod +x gradlew

# 下载新版 Gradle (系统自带版本太旧)
echo "=== 安装 Gradle 8.2 ==="
wget -q https://services.gradle.org/distributions/gradle-8.2-bin.zip
unzip -q -o gradle-8.2-bin.zip
export PATH=$PWD/gradle-8.2/bin:$PATH

echo "Gradle 版本: $(gradle --version | head -3)"

# 编译 Debug APK
echo ""
echo "=== 编译 Android APK (Debug) ==="
gradle assembleDebug --no-daemon

echo ""
echo "=== Debug 编译完成 ==="

In [ ]:
%%bash
cd /content/ech-workers-NG
APK_PATH="app/build/outputs/apk/debug/app-arm64-v8a-debug.apk"

if [ -f "$APK_PATH" ]; then
    echo "✅ Debug APK 编译成功！"
    ls -lh $APK_PATH
else
    echo "❌ APK 编译失败"
    find . -name "*.apk" 2>/dev/null
fi

## 7. 编译 Release APK (签名版本)

> **签名配置说明**：
> - 下方设置签名密钥的密码和别名
> - 如果没有现有密钥，会自动生成一个新的签名密钥

In [ ]:
# ============ 签名密钥配置 ============
# 修改以下变量来设置你的签名密钥信息

KEYSTORE_PASSWORD = "echworkers123"  # 密钥库密码
KEY_ALIAS = "echworkers"              # 密钥别名
KEY_PASSWORD = "echworkers123"        # 密钥密码

# 保存到环境变量供后续使用
import os
os.environ['KEYSTORE_PASSWORD'] = KEYSTORE_PASSWORD
os.environ['KEY_ALIAS'] = KEY_ALIAS
os.environ['KEY_PASSWORD'] = KEY_PASSWORD

print(f"✅ 签名配置已设置")
print(f"   密钥别名: {KEY_ALIAS}")

In [ ]:
%%bash
cd /content/ech-workers-NG

# 生成签名密钥（如果不存在）
KEYSTORE_FILE="/content/release-key.jks"

if [ ! -f "$KEYSTORE_FILE" ]; then
    echo "=== 生成签名密钥 ==="
    keytool -genkey -v \
        -keystore $KEYSTORE_FILE \
        -keyalg RSA \
        -keysize 2048 \
        -validity 10000 \
        -alias $KEY_ALIAS \
        -storepass $KEYSTORE_PASSWORD \
        -keypass $KEY_PASSWORD \
        -dname "CN=ECH Workers, OU=Dev, O=ECH, L=Unknown, ST=Unknown, C=CN"
    echo "✅ 签名密钥已生成: $KEYSTORE_FILE"
else
    echo "✅ 使用现有签名密钥: $KEYSTORE_FILE"
fi

In [ ]:
%%bash
cd /content/ech-workers-NG
export PATH=$PWD/gradle-8.2/bin:$PATH

KEYSTORE_FILE="/content/release-key.jks"

echo "=== 编译 Release APK (各架构分离) ==="

# 使用命令行参数传递签名信息
gradle assembleRelease --no-daemon \
    -Pandroid.injected.signing.store.file=$KEYSTORE_FILE \
    -Pandroid.injected.signing.store.password=$KEYSTORE_PASSWORD \
    -Pandroid.injected.signing.key.alias=$KEY_ALIAS \
    -Pandroid.injected.signing.key.password=$KEY_PASSWORD

echo ""
echo "=== Release APK 列表 ==="
find app/build/outputs/apk/release -name "*.apk" -exec ls -lh {} \; 2>/dev/null || echo "未找到 Release APK"

## 8. 下载 APK

In [ ]:
# 下载 Debug APK
from google.colab import files
import os, shutil

debug_apk = "/content/ech-workers-NG/app/build/outputs/apk/debug/app-debug.apk"
if os.path.exists(debug_apk):
    shutil.copy(debug_apk, "/content/EchWorkers-debug.apk")
    print("正在下载 Debug APK...")
    files.download("/content/EchWorkers-debug.apk")
else:
    print("Debug APK 不存在，请先运行 Debug 编译")

In [ ]:
# 下载 Release APK (arm64-v8a 版本)
from google.colab import files
import os, shutil

release_dir = "/content/ech-workers-NG/app/build/outputs/apk/release"
arm64_apk = None

# 查找 arm64-v8a 版本
if os.path.exists(release_dir):
    for f in os.listdir(release_dir):
        if "arm64-v8a" in f and f.endswith(".apk"):
            arm64_apk = os.path.join(release_dir, f)
            break

if arm64_apk and os.path.exists(arm64_apk):
    shutil.copy(arm64_apk, "/content/EchWorkers-arm64-v8a-release.apk")
    print("正在下载 Release APK (arm64-v8a)...")
    files.download("/content/EchWorkers-arm64-v8a-release.apk")
else:
    print("arm64-v8a Release APK 不存在，请先运行 Release 编译")
    print("可用的 APK:")
    if os.path.exists(release_dir):
        for f in os.listdir(release_dir):
            if f.endswith(".apk"):
                print(f"  - {f}")

In [ ]:
# 下载所有 Release APK (打包)
from google.colab import files
import os, shutil

os.makedirs("/content/apk_output", exist_ok=True)

release_dir = "/content/ech-workers-NG/app/build/outputs/apk/release"
if os.path.exists(release_dir):
    for f in os.listdir(release_dir):
        if f.endswith(".apk"):
            shutil.copy(os.path.join(release_dir, f), f"/content/apk_output/{f}")
            print(f"✅ 已复制: {f}")
    
    # 打包
    !cd /content/apk_output && zip -r /content/EchWorkers-Release-All.zip *.apk
    print("\n正在下载所有 Release APK...")
    files.download("/content/EchWorkers-Release-All.zip")
else:
    print("Release APK 不存在，请先运行 Release 编译")

---
## 调试

In [ ]:
!echo "=== Go ==="
!go version
!echo "=== NDK ==="
!ls $ANDROID_HOME/ndk/ 2>/dev/null || echo "未安装"
!echo "=== gomobile ==="
!which gomobile
!echo "=== Gradle ==="
!ls /content/ech-workers-NG/gradle-8.2/bin/gradle 2>/dev/null && echo "Gradle 8.2 已安装" || echo "Gradle 未安装"
!echo "=== core.aar ==="
!ls -lh /content/ech-workers-NG/app/libs/*.aar 2>/dev/null || echo "不存在"
!echo "=== APK 文件 ==="
!find /content/ech-workers-NG/app/build/outputs/apk -name "*.apk" 2>/dev/null || echo "无 APK"